# SECULOCK: SMART LOCKER SYSTEM

This project implements a smart locker security system using Raspberry Pi, Face Recognition, OTP Verification, and Intrusion Detection Mechanisms. The system ensures secure access control by verifying the user's identity through facial recognition using the DeepFace library and the FaceNet model. If face recognition fails, an OTP (One-Time Password) is sent to the owner's registered email for secondary authentication.

Additionally, the system incorporates vibration sensors to detect unauthorized access attempts. If a security breach is detected, an alert email with an intruder's photo is sent to the owner, and a buzzer is activated to deter the intruder.

## Key Features

✅ Face Recognition using DeepFace (FaceNet)

✅ OTP-based Authentication in case of face mismatch

✅ Intrusion Detection with vibration sensors

✅ Email Alert with intruder photo for unauthorized access

✅ LCD Display for real-time system status updates

This Jupyter Notebook contains the Python implementation of the SecuLock system, interfacing various hardware components like Raspberry Pi, USB Camera, 16x2 I2C LCD, 4x4 Matrix Keypad, Vibration Sensors, and Relays for real-time security monitoring.

## Detailed Explanation of Code

### Importing Libraries
This section imports all the necessary Python libraries used throughout the project for functionalities like image processing, face recognition, email communication, GPIO pin control, I2C-based LCD interaction, and system cleanup.

In [ ]:
import json                 # For handling JSON data structures if needed
import cv2                  # OpenCV library for image processing and face capture
import numpy as np          # Useful for numerical operations (used in image handling)
import time                 # Provides delay/timer functions like time.sleep()
import smtplib              # For sending emails via SMTP (used for OTP & alerts)
import random               # To generate random OTPs
import lgpio as GPIO        # Library for GPIO control using lgpio (better for multitasking than RPi.GPIO)
import smbus2 as smbus      # For I2C communication with devices like the LCD
from deepface import DeepFace    # Face recognition using FaceNet model
from email.mime.text import MIMEText              # For plain text email content
from email.mime.multipart import MIMEMultipart    # For email with attachments
from email.mime.base import MIMEBase              # Base class for attachments
from email import encoders                        # For encoding attachments
import atexit               # To register cleanup functions on program exit

### Email Configuration
This section sets up the email parameters including SMTP server details, sender and receiver email addresses, authentication password, and the default image used for face verification.

In [ ]:
# SMTP configuration
smtp_port = 587  # Port for TLS encryption
smtp_server = "smtp.gmail.com"  # Gmail's SMTP server

# Sender and receiver information
email_from = "smartlockersender@gmail.com"  
email_to = "owner@gmail.com" 

pswd = "*********" # App-specific password for the sender email (generated from Google account settings)
subject = "Theft detected !!"  # Subject of the alert email

SOURCE_IMAGE = "owner.jpeg"  # Authorized user's reference image


### LCD Display Setup
This section initializes the I2C communication for a 16x2 LCD display, setting up the necessary parameters such as I2C address, command/data modes, line addresses, and control bits.

In [ ]:
# I2C address of the LCD
I2C_ADDR = 0x27  

# Initialize I2C bus (1 indicates /dev/i2c-1 on Raspberry Pi)
bus = smbus.SMBus(1)

# LCD command/data flags
LCD_CHR = 1  # Mode - Sending data to display
LCD_CMD = 0  # Mode - Sending command to LCD

# LCD RAM addresses for each line
LINE_1 = 0x80  # Address of the first line
LINE_2 = 0xC0  # Address of the second line

# LCD control bits
ENABLE = 0b00000100     # Enable bit
BACKLIGHT = 0b00001000  # Backlight control bit

### LCD Control Functions
This section defines the core functions needed to operate an I2C-based LCD. It includes sending commands/data, toggling the enable pin, initializing the LCD, and displaying messages.

In [ ]:
# Sends data or commands to the LCD
def lcd_write(bits, mode):
    try:
        # Extract high and low 4 bits from the byte and add mode & backlight
        high_bits = mode | (bits & 0xF0) | BACKLIGHT
        low_bits = mode | ((bits << 4) & 0xF0) | BACKLIGHT

        # Write high bits to LCD
        bus.write_byte(I2C_ADDR, high_bits)
        lcd_toggle_enable(high_bits)
        time.sleep(0.0005)

        # Write low bits to LCD
        bus.write_byte(I2C_ADDR, low_bits)
        lcd_toggle_enable(low_bits)
        time.sleep(0.0005)
    except Exception as e:
        print("LCD write error:", e)

# Triggers the LCD enable pin to latch data
def lcd_toggle_enable(bits):
    time.sleep(0.0005)
    bus.write_byte(I2C_ADDR, bits | ENABLE)     # Set ENABLE high
    time.sleep(0.0005)
    bus.write_byte(I2C_ADDR, bits & ~ENABLE)    # Set ENABLE low
    time.sleep(0.0005)

# Initializes the LCD with standard settings
def lcd_init():
    try:
        time.sleep(0.1)  # Wait for power up
        lcd_write(0x33, LCD_CMD)  # Initialization
        time.sleep(0.005)
        lcd_write(0x32, LCD_CMD)  # 4-bit mode
        time.sleep(0.005)
        lcd_write(0x28, LCD_CMD)  # 2-line display, 5x8 dots
        time.sleep(0.005)
        lcd_write(0x0C, LCD_CMD)  # Display ON, cursor OFF
        time.sleep(0.005)
        lcd_write(0x06, LCD_CMD)  # Cursor moves to right
        time.sleep(0.005)
        lcd_write(0x01, LCD_CMD)  # Clear screen
        time.sleep(0.005)
    except Exception as e:
        print("LCD initialization error:", e)

# Displays text on the specified LCD line (LINE_1 or LINE_2)
def lcd_display(text, line):
    lcd_write(line, LCD_CMD)  # Set line address
    for char in text.ljust(16):  # Pad or truncate to 16 characters
        lcd_write(ord(char), LCD_CHR)

# Initialize the LCD
lcd_init()

### Email Sending and OTP Generation Functions
This section handles sending alert emails with captured intruder images and generating/sending OTPs to the user's registered email.

In [ ]:
# Sends an alert email with the captured face image attached
def send_emails(email_to):
    try:
        body = "Unauthorized access attempt detected."  # Email body text

        msg = MIMEMultipart()  # Create multipart email container
        msg['From'], msg['To'], msg['Subject'] = email_from, email_to, subject  # Set email headers

        msg.attach(MIMEText(body, 'plain'))  # Attach plain text body

        filename = "captured_face.jpg"  # Image to attach
        with open(filename, 'rb') as attachment:
            attachment_package = MIMEBase('application', 'octet-stream')  # Prepare attachment container
            attachment_package.set_payload(attachment.read())  # Read image content
            encoders.encode_base64(attachment_package)  # Encode for safe transmission
            attachment_package.add_header('Content-Disposition', f"attachment; filename= {filename}")  # Attachment header
            msg.attach(attachment_package)  # Attach image to email

        with smtplib.SMTP(smtp_server, smtp_port) as server:  # Connect to SMTP server
            server.starttls()  # Start TLS encryption
            server.login(email_from, pswd)  # Login to sender email
            server.sendmail(email_from, email_to, msg.as_string())  # Send email
            print(f"Email sent to: {email_to}")  # Confirm sending

    except Exception as e:
        print("Email sending error:", e)  # Print error if any


# Sends an OTP email without attachments
def otp_sent(subject, body):
    try:
        msg = MIMEText(body)  # Create plain text message
        msg['From'], msg['To'], msg['Subject'] = email_from, email_to, subject  # Set headers

        with smtplib.SMTP(smtp_server, smtp_port) as server:  # Connect to SMTP server
            server.starttls()  # Secure connection
            server.login(email_from, pswd)  # Login sender email
            server.send_message(msg)  # Send OTP email

    except Exception as e:
        print("Email error:", e)  # Print error if any


# Generates and sends a 6-digit OTP, then returns it
def send_otp():
    otp = str(random.randint(100000, 999999))  # Generate random 6-digit OTP
    otp_sent("Your OTP for Verification", f"Your OTP is: {otp}")  # Send OTP email
    return otp  # Return OTP string

### Face Capture and Comparison
This section captures a face from a video frame and compares it with a stored source image using the FaceNet model to verify identity.

In [ ]:
# Capture the first detected face from the frame and save it as an image file
def capture_face(frame):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml") # Load face detector
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert frame to grayscale for detection
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))  # Detect faces

    if len(faces) > 0:  # If at least one face detected
        x, y, w, h = faces[0]  # Get coordinates of first face
        face_roi = frame[y:y + h, x:x + w]  # Extract face region from frame
        path = "captured_face.jpg"  # Define path to save face image
        cv2.imwrite(path, face_roi)  # Save face image to disk
        return path  # Return saved image path
    return None  # Return None if no face detected


# Compare the saved face image with the source image using DeepFace FaceNet model
def compare_faces(source, captured):
    try:
        # Verify if two images are of the same person using FaceNet model
        result = DeepFace.verify(img1_path=source, img2_path=captured, model_name="FaceNet", enforce_detection=True)
        return result.get('verified', False)  # Return True if verified, else False
    except Exception as e:
        print("Face comparison error:", e)  # Print error if comparison fails
        return False  # Return False on error

### GPIO Setup and Cleanup
This section initializes the GPIO chip for controlling hardware pins and ensures resources are properly released when the program exits.

In [ ]:
chip = GPIO.gpiochip_open(0)  # Open GPIO chip 0 for pin control

def cleanup():
    print("Releasing GPIO resources...")  
    GPIO.gpiochip_close(chip)  # Close GPIO chip to free resources

atexit.register(cleanup)  # Register cleanup() to run automatically on program exit

### Keypad Initialization and Reading
This section configures GPIO pins for a 4x4 keypad matrix and defines a function to detect which key is pressed by scanning rows and columns.

In [ ]:
ROWS = [17, 27, 22, 10]  # GPIO pins connected to keypad rows
COLS = [9, 11, 5, 6]     # GPIO pins connected to keypad columns

KEYPAD = [               # Key layout corresponding to rows and columns
    ['1', '2', '3', 'A'],
    ['4', '5', '6', 'B'],
    ['7', '8', '9', 'C'],
    ['*', '0', '#', 'D']
]

# Configure row pins as outputs (initially HIGH)
for row in ROWS:
    GPIO.gpio_claim_output(chip, row, 1)  # Claim output with initial HIGH

# Configure column pins as inputs with pull-up resistors
for col in COLS:
    GPIO.gpio_claim_input(chip, col, GPIO.SET_PULL_UP)  # Claim input with pull-up

def read_keypad():
    for row_index, row_pin in enumerate(ROWS):
        GPIO.gpio_write(chip, row_pin, 0)  # Set current row LOW to detect key press
        for col_index, col_pin in enumerate(COLS):
            if GPIO.gpio_read(chip, col_pin) == 0:  # If column reads LOW, key pressed
                time.sleep(0.2)  # Debounce delay to avoid multiple detections
                GPIO.gpio_write(chip, row_pin, 1)  # Reset row to HIGH
                return KEYPAD[row_index][col_index]  # Return pressed key character
        GPIO.gpio_write(chip, row_pin, 1)  # Reset row to HIGH if no key press detected
    return None  # Return None if no key pressed

### GPIO Pin Initialization for Relays and Sensors
This section sets up GPIO pins for controlling the relay (lock and buzzer) as outputs and for reading vibration sensors as inputs with pull-up resistors.

In [ ]:
RELAY_LOCK_PIN, RELAY_BUZZER_PIN = 26, 19  # GPIO pins for lock relay and buzzer relay
VIBRATION_SENSOR_1 = 23  # GPIO pin connected to Vibration Sensor 1
VIBRATION_SENSOR_2 = 24  # GPIO pin connected to Vibration Sensor 2

# Configure vibration sensors as input with pull-up resistors
GPIO.gpio_claim_input(chip, VIBRATION_SENSOR_1, GPIO.SET_PULL_UP)  # Vibration Sensor 1 input
GPIO.gpio_claim_input(chip, VIBRATION_SENSOR_2, GPIO.SET_PULL_UP)  # Vibration Sensor 2 input

# Configure relay pins as output and set initial value to HIGH (1)
GPIO.gpio_claim_output(chip, RELAY_LOCK_PIN, 1)    # Lock relay output pin
GPIO.gpio_claim_output(chip, RELAY_BUZZER_PIN, 1)  # Buzzer relay output pin

### Buzzer Control and Vibration Sensor Monitoring
This section defines functions to activate the buzzer for a specified duration and to monitor vibration sensors. If any sensor detects vibration (active low), the buzzer is triggered and an alert email is sent.

In [ ]:
# Activate the buzzer for a given duration (default 5 seconds)
def trigger_buzzer(duration=5):
    GPIO.gpio_write(chip, RELAY_BUZZER_PIN, 0)  # Turn buzzer ON (active low)
    time.sleep(duration)                         # Wait for specified duration
    GPIO.gpio_write(chip, RELAY_BUZZER_PIN, 1)  # Turn buzzer OFF

# Monitor vibration sensors and respond if triggered
def vibra():
    sensor_1_state = GPIO.gpio_read(chip, VIBRATION_SENSOR_1)  # Read vibration sensor 1
    sensor_2_state = GPIO.gpio_read(chip, VIBRATION_SENSOR_2)  # Read vibration sensor 2

    if sensor_1_state == 0 or sensor_2_state == 0:             # If any sensor is activated (LOW)
        trigger_buzzer()                                       # Activate buzzer alert
        otp_sent("Theft Detected", "Vibration sensors activated")  # Send theft alert email
        time.sleep(0.1)                                        # Short delay to debounce/smooth sensor reading

### I2C Bus Reset Function
This function attempts to reset the I2C communication bus by closing and reopening it, helping to recover from any communication errors.

In [ ]:
def reset_i2c():
    try:
        bus = smbus.SMBus(1)    # Open I2C bus 1
        bus.close()             # Close the bus to reset connection
        time.sleep(1)           # Wait for 1 second before reopening
        bus = smbus.SMBus(1)    # Reopen the I2C bus
        print("I2C reset successful")  # Inform successful reset
    except Exception as e:
        print("I2C reset failed:", e)  # Print error message if reset fails

### Main Loop
This is the continuous running loop controlling the locker system. It waits for user input to start face recognition, handles face verification, OTP fallback, and locker control, while updating the LCD and managing hardware pins.

In [ ]:
while True:
    # Reset I2C bus each loop to avoid communication issues
    reset_i2c()
    
    try:
        lcd_display("Press A ", LINE_1)          # Prompt user to press 'A' to start
        lcd_display("to start", LINE_2)
        i = read_keypad()                        # Read keypad input
    except Exception as e:
        print("LCD Error:", e)
        continue                                # Retry if LCD or keypad fails

    if i == "A":                                # If 'A' pressed, begin face recognition
        cap = cv2.VideoCapture(0)               # Initialize webcam capture

        while cap.isOpened():
            try:
                # Clear LCD and show welcome message
                lcd_display("", LINE_1)
                lcd_display("", LINE_2)
                lcd_display("SecuLock", LINE_1)
                time.sleep(5)

                ret, frame = cap.read()          # Capture video frame
                if not ret:
                    break                       # Stop if frame capture fails

                captured_path = capture_face(frame)  # Detect and save face
                lcd_display("Face Captured", LINE_1)
                print("Face captured")

                # Compare captured face with authorized image
                if captured_path and compare_faces(SOURCE_IMAGE, captured_path):
                    lcd_display("Access Granted", LINE_1)  # Success feedback
                    GPIO.gpio_write(chip, RELAY_LOCK_PIN, 0)  # Unlock relay
                    time.sleep(1)

                    # Wait for user to close locker with 'B'
                    while True:
                        try:
                            lcd_display("Press B", LINE_1)
                            lcd_display("to lock", LINE_2)
                            close = read_keypad()
                        except Exception as e:
                            print("LCD Error:", e)
                            continue

                        if close == "B":           # Close locker command
                            lcd_display("Closed", LINE_1)
                            GPIO.gpio_write(chip, RELAY_LOCK_PIN, 1)  # Lock relay
                            break                   # Exit locker close loop
                    break                           # Exit face recognition loop

                else:
                    # Face mismatch: send OTP for verification
                    lcd_display("Face not matched", LINE_1)
                    sent_otp = send_otp()
                    lcd_display("OTP Sent", LINE_1)
                    print("OTP sent")
                    time.sleep(1)
                    lcd_display("Enter OTP:", LINE_1)

                    input_sequence = ""             # Store entered OTP digits
                    otp_verified = False            # OTP verification flag

                    while True:
                        try:
                            key = read_keypad()     # Read keypad inputs
                        except Exception as e:
                            print("Keypad Error:", e)
                            continue

                        if key:
                            if key == "#":          # Submit OTP for verification
                                if input_sequence == sent_otp:
                                    lcd_display("Access Granted", LINE_1)
                                    time.sleep(1)
                                    lcd_display("", LINE_1)
                                    lcd_display("", LINE_2)
                                    GPIO.gpio_write(chip, RELAY_LOCK_PIN, 0)  # Unlock relay
                                    otp_verified = True
                                else:
                                    lcd_display("Access Denied", LINE_1)
                                    trigger_buzzer()          # Alert on failure
                                    send_emails(email_to)     # Send alert email
                                break                  # Exit OTP input loop

                            elif key == "*":         # Clear OTP input
                                input_sequence = ""
                            else:
                                input_sequence += key # Append digit to OTP input
                                lcd_display(input_sequence, LINE_2)
                                print(key)

                    # If OTP verified, allow user to close locker
                    if otp_verified:
                        while True:
                            try:
                                lcd_display("Press B", LINE_1)
                                lcd_display("to lock", LINE_2)
                                close2 = read_keypad()
                            except Exception as e:
                                print("LCD Error:", e)
                                continue

                            if close2 == "B":       # Close locker command
                                lcd_display("Closed", LINE_1)
                                GPIO.gpio_write(chip, RELAY_LOCK_PIN, 1)  # Lock relay
                                break                   # Exit locker close loop
                    break                       # Exit face recognition loop

            except Exception as e:
                print("Error during face recognition loop:", e)

        cap.release()                   # Release webcam resource
        cv2.destroyAllWindows()         # Close any OpenCV windows